In [1]:
import os
import numpy as np
import torch 
from PIL import Image

自定义数据集
需要实现torch.utils.data.Dataset的类，并且实现两个函数：
- __len__
- __getitem__

其中__geititem__需要返回如下参数：
- image: PIL格式的图像(H,W)
- target: 字典类型的字段：
    - boxes(FloatTensor[N, 4]): bounding box的坐标[x0,y0,x1,y1]
    - labels (Int64Tensor[N]): 每个bounding box的标签，0表示背景
    - image_id (Int64Tensor[1])：图片的ID标志，这是图片与图片之间的唯一的区分，在模型的验证阶段需要
    - area (Tensor[N]):bounding box的面积，COCO一种衡量的标准，用来衡量大，中，小bounding box的得分
    - iscrowd (UInt8Tensor[N])： iscrowd = True的实例在评估期间将被忽略。
    - masks (UInt8Tensor[N, H, W])：每个对象的分割蒙版
    - keypoints (FloatTensor[N, K, 3])：对于 N 个对象中的每一个，它包含[x, y, visibility]格式的 K 个关键点，以定义对象。


In [2]:
class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms

        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
    
    def __getitem__(self, idx):
        # 加载图片和掩码标签
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")

        mask = Image.open(mask_path)
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]

        masks = mask == obj_ids[:, None, None]

        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target={}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)


        return img, target
    
    def __len__(self):
        return len(self.imgs)



FineTuning的方式

In [3]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# 加载一个由COCO数据集与训练好的模型
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 替换模型中的分类的种类
# 这里设置为2，1：表示person类，其他的表示背景
num_classes = 2

#获得输入的类别的特征值
in_features = model.roi_heads.box_predictor.cls_score.in_features
# 替换预训练的头部
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\IKAS/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100.0%


使用不同的backbone的方式来修改模型

In [4]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator


backbone = torchvision.models.mobilenet_v2(pretrained=True).features

backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32,64,128,256,512),),
                                    aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

model = FasterRCNN(backbone,
                    num_classes=2,
                    rpn_anchor_generator=anchor_generator,
                    box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\IKAS/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):

    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                        hidden_layer,
                                                        num_classes)


    return model

In [6]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

ModuleNotFoundError: No module named 'transforms'

In [7]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

NameError: name 'get_transform' is not defined